![image](https://miro.medium.com/max/1384/1*7R068tzqqK-1edu4hbAVZQ.png)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import glob as gb
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping ,ReduceLROnPlateau , ModelCheckpoint
from keras.applications.mobilenet import MobileNet ,preprocess_input

In [ ]:
img=cv2.imread('../input/pest-dataset/pest/train/aphids/jpg_0 - Copy (2).jpg')
plt.imshow(img)
plt.show()


In [ ]:
img=np.array(img)
img.shape

In [ ]:
pip install gdown

In [ ]:
import gdown

In [ ]:
#https://drive.google.com/drive/folders/1T8jcvH-bRTywo2OmW9xcgTn5ShIRrCU3?usp=sharing
#https://drive.google.com/drive/folders/1j9ONhi6CFw41yjpaXTLViOu_yhzUKuUr?usp=sharing
url="https://drive.google.com/uc?id=1j9ONhi6CFw41yjpaXTLViOu_yhzUKuUr"
output='kaggle_output_train.zip'
gdown.download(url,output,quiet=False)

In [ ]:
train_dir='../input/pest-dataset/pest/train/'
test_dir='../input/pest-dataset/pest/test/'

training=tf.keras.preprocessing.image.ImageDataGenerator(
    zca_epsilon=1e-06,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.05
).flow_from_directory(train_dir,batch_size=16,target_size=(224,224),subset="training")

validing=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.05
).flow_from_directory(train_dir,batch_size=16,target_size=(224,224),subset='validation',shuffle=True)

testing=tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
).flow_from_directory(test_dir,batch_size=16,target_size=(224,224),shuffle=True)


In [ ]:
mobilenet=MobileNet(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [ ]:
#optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.99)
optimizer=Adam(lr=0.009,beta_1=0.9,beta_2=0.99)

In [ ]:
EarlyStop=EarlyStopping(patience=10,restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_acc',verbose=2,factor=0.5,min_lr=0.00001)
callback=[EarlyStop , Reduce_LR]
#model_check=ModelCheckpoint('model.hdf5',monitor='val_loss',verbose=1,save_best_only=True)
#callback=[EarlyStop , Reduce_LR,model_check]

In [ ]:
mobilenet.trainable=False

In [ ]:
model=Sequential([
    mobilenet,
    MaxPooling2D(3,2),
    Flatten(),
    #Dense(128,activation='relu'),
    Dense(512,activation='relu'),
    BatchNormalization(),
    Dense(1024,activation='relu'),
    BatchNormalization(),
    Dense(512,activation='relu'),
    BatchNormalization(),
    Dense(9,activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
#history=model.fit(training,validation_data=validing,epochs=20,batch_size=16,
#                 steps_per_epoch=len(training) // 16,validation_steps=len(validing) // 8,
#                 callbacks=callback, verbose=2)

history=model.fit(training,validation_data=validing,epochs=20,batch_size=16,
                  steps_per_epoch=len(training),validation_steps=len(validing),
                  verbose=2)

In [ ]:
loss , acc = model.evaluate(testing)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
img.shape

In [ ]:
img = cv2.resize(img,(224,224))     # resize image to match model's expected sizing
img = img.reshape(1,224,224,3)

In [ ]:
model.predict(img)

In [ ]:
from IPython.display import FileLink
FileLink(r'outputname.tar.gz')

In [ ]:
from IPython.display import FileLink
FileLink(r'outputname.h5')

In [ ]:
from IPython.display import FileLink
FileLink(r'processed_file.zip')


In [ ]:
l=['aphids','armyworm','beetle','bollworm','grasshopper','mites','mosquito','sawfly','stem borer']